# Modeling Time Series - Univariate and ML methods

In [ ]:
!pip install -r requirements.txt --upgrade --quiet

In [ ]:
import pandas as pd

In [ ]:
flights_train = pd.read_csv('data/splits/holout_train.csv',
                      index_col=0, 
                      parse_dates=True)

flights_test = pd.read_csv('data/splits/holout_test.csv',
                           index_col=0, 
                           parse_dates=True)


In [ ]:
flights_train.index = pd.to_datetime(flights_train.index)
flights_test.index = pd.to_datetime(flights_test.index)

## Prepare data

In [ ]:
y_train = flights_train['Passengers']
y_test = flights_test['Passengers']

In [ ]:
flights_train

In [ ]:
X_train = flights_train[['log','Lag_1', 'Lag_2', 'Lag_3', 
         'Lag_12', 'Lag_24', 'Lag_36']]

X_test = flights_test[['log','Lag_1', 'Lag_2', 'Lag_3', 
         'Lag_12', 'Lag_24', 'Lag_36']]

## Forecast Horizon Selection

In [ ]:
FORECAST_HORIZON = 24
SEAS_PERIODS = 12

## Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
es_model = ExponentialSmoothing(
    y_train, 
    trend='mul', 
    seasonal='mul', 
    seasonal_periods=SEAS_PERIODS)

In [ ]:
from timeseries.styler import style_dataframe
es_fit = es_model.fit()
fit_params = es_fit.params
es_summary_df = pd.DataFrame(fit_params.items(), columns=['Parameter', 'Value'])

In [ ]:
style_dataframe(es_summary_df.head())

In [ ]:
es_forecast = es_fit.predict(start=y_test.index[0], 
                             end=y_test.index[-1])


## SARIMA with hard-coded parameters

In [ ]:
import statsmodels.api as sm
from timeseries.metrics import compute_metrics
order = (1, 1, 1)  
seasonal_order = (1, 1, 1, 12)


In [ ]:
model = sm.tsa.SARIMAX(y_train, 
                       order=order, 
                       seasonal_order=seasonal_order)
fit = model.fit()

In [ ]:
sarima_base_forecast = fit.get_forecast(steps=len(y_test))

In [ ]:
sarima_base_test_preds = sarima_base_forecast.predicted_mean

## ARIMA with Grid Hyperparameters

In [ ]:
%%writefile timeseries/sarima_auto.py
import itertools
import statsmodels.api as sm
import numpy as np
import warnings
from tqdm import tqdm
import pandas as pd

def search_arima_orders(y_train, 
                        s=12,
                        p_range=range(0, 3), d_range=range(0, 2), q_range=range(0, 3),
                        P_range=range(0, 2), D_range=range(0, 2), Q_range=range(0, 2),
                        period_str="M", 
                        verbose=True,
                        top_n=10):


    if not isinstance(y_train.index, (pd.PeriodIndex, pd.DatetimeIndex)):
        raise ValueError("Index must be a PeriodIndex or DatetimeIndex.")
    if y_train.index.freq is None:
        try:
            y_train.index = y_train.index.to_period(period_str)
        except:
            y_train.index.freq = period_str

    use_seasonal = len(y_train) >= 2 * s
    results = []

    orders = list(itertools.product(p_range, d_range, q_range))
    seasonal_orders = [(0, 0, 0)] if not use_seasonal else list(itertools.product(P_range, D_range, Q_range))

    total_combos = len(orders) * len(seasonal_orders)
    if verbose:
        print(f"🔍 Evaluating {total_combos} (p,d,q)(P,D,Q) combinations.")

    for order in tqdm(orders, desc="Grid Searching", disable=not verbose):
        for seasonal in seasonal_orders:
            seasonal_order = (seasonal[0], seasonal[1], seasonal[2], s) if use_seasonal else (0, 0, 0, 0)

            try:
                    model = sm.tsa.statespace.SARIMAX(
                        y_train,
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False
                    )
                    result = model.fit(disp=False)
                    results.append({
                        'order': order,
                        'seasonal_order': seasonal_order,
                        'aic': result.aic,
                        'bic': result.bic
                    })
            except Exception:
                continue

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='aic').reset_index(drop=True)

    if verbose:
        print("Top Models by AIC:")
        print(results_df.head(top_n))

    return results_df


In [ ]:
from timeseries.arima import search_arima_orders

In [ ]:
arima_orders_results_df = search_arima_orders(y_train, 
                                 s=12, 
                                 verbose=False, 
                                 top_n=20)

In [ ]:
style_dataframe(arima_orders_results_df.head())

In [ ]:
chosen_order = arima_orders_results_df.iloc[0]['order']
chosen_seasonal = arima_orders_results_df.iloc[0]['seasonal_order']

In [ ]:
SARIMA_tuned_model = sm.tsa.SARIMAX(y_train, 
                                    order=chosen_order, 
                                    seasonal_order=chosen_seasonal)
fit = SARIMA_tuned_model.fit()

In [ ]:
forecast = fit.get_forecast(steps=len(y_test))
arima_tuned_forecast_mean = forecast.predicted_mean

## ML Approaches on Lagged Features

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

In [ ]:
rf = RandomForestRegressor(n_estimators=100, 
                           random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [ ]:
xgb = XGBRegressor(n_estimators=100, 
                   learning_rate=0.1, 
                   max_depth=3, 
                   random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

## Prophet 

In [ ]:
from prophet import Prophet
import pandas as pd

y_train_prophet = y_train.copy()
if isinstance(y_train_prophet.index, pd.PeriodIndex):
    y_train_prophet.index = y_train_prophet.index.to_timestamp()

In [ ]:
df_prophet = y_train_prophet.reset_index()
df_prophet.columns = ['ds', 'y']

In [ ]:
help(Prophet)

In [ ]:
model = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)
model.fit(df_prophet)

In [ ]:
future = model.make_future_dataframe(periods=len(y_test), freq='M')  
prophet_forecast = model.predict(future)
prophet_fc = prophet_forecast['yhat']

## Combine training forecasts together

In [ ]:
combined_forecasts = {
    'exponential_smoothing': es_forecast,
    'sarima_tuned': arima_tuned_forecast_mean,
    'sarima_base': sarima_base_test_preds,
    'random_forest': rf_pred,
    'xgboost': xgb_pred,
    'prophet': prophet_fc
}

In [ ]:
import pandas as pd

def combine_and_evaluate_forecasts(y_true, forecast_dict, 
                                   compute_metrics_fn, 
                                   sort_by='RMSE'):
    
    combined = {'actual': y_true.values}
    index = y_true.index

    for name, preds in forecast_dict.items():
        preds = pd.Series(preds)
        if len(preds) != len(y_true):
            preds = preds[-len(y_true):]  
        preds.index = index
        combined[name] = preds.values

    combined_df = pd.DataFrame(combined, index=index)

    metrics_records = []
    for model_name in forecast_dict.keys():
        y_pred = combined_df[model_name]
        metrics = compute_metrics_fn(combined_df['actual'], y_pred)
        metrics['Model'] = model_name
        metrics_records.append(metrics)

    metrics_df = pd.DataFrame(metrics_records).set_index('Model')
    best_model = metrics_df[sort_by].idxmin() if sort_by in metrics_df.columns else None
    return combined_df, metrics_df, best_model

In [ ]:
EVAL_METRIC = 'RMSE'

combined_df, metrics_df, best_model = combine_and_evaluate_forecasts(
    y_test,
    combined_forecasts,
    compute_metrics_fn=compute_metrics,  
    sort_by=EVAL_METRIC
)

print(f"🏆 Best model by {EVAL_METRIC}:{best_model}")

In [ ]:
combined_df.head()

In [ ]:
style_dataframe(metrics_df.drop(columns='R²'))

## Plot Forecast Plots

In [ ]:
from modelviz.forecast import plot_forecast_subplots

In [ ]:
help(plot_forecast_subplots)

In [ ]:
plot_forecast_subplots(y_train, y_test, combined_df)

## Estimating with Forecast Horizon


### Index readjustment

In [ ]:
def ensure_datetime_index(series):
    if isinstance(series.index, pd.PeriodIndex):
        series.index = series.index.to_timestamp()
    elif not isinstance(series.index, pd.DatetimeIndex):
        series.index = pd.to_datetime(series.index)
    return series

In [ ]:
y_train = ensure_datetime_index(y_train)
y_test = ensure_datetime_index(y_test)
y_full = pd.concat([y_train, y_test]).sort_index()
y_full = y_full.asfreq(pd.infer_freq(y_full.index))
len(y_full)


### Fit on full dataset (train and test combined)

In [ ]:
arima_tune_fitted = sm.tsa.SARIMAX(
    y_full, 
    order=chosen_order, 
    seasonal_order=chosen_seasonal
).fit()


### Make predictions into the future

In [ ]:
arima_tune_fitted.get_forecast(steps=FORECAST_HORIZON).predicted_mean

In [ ]:
arima_tune_fitted.get_forecast(steps=FORECAST_HORIZON).conf_int()

## Visualize forecast

In [ ]:
from modelviz.forecast import plot_forecast

In [ ]:
forecast_df = plot_forecast(
    data=y_full,
    model=arima_tune_fitted,
    horizon=FORECAST_HORIZON,
    model_type='statsmodels',
    alpha=0.05,
    plot=True,
    ci_color='grey',
    forecast_color='black',
    historical_color='grey'
)

In [ ]:
forecast_df = plot_forecast(
    data=y_full,
    model=arima_tune_fitted,
    horizon=200,
    model_type='statsmodels',
    alpha=0.05,
    plot=True,
    ci_color='grey',
    forecast_color='black',
    historical_color='grey'
)